<a href="https://colab.research.google.com/github/Kethanvr/Reag-VecotrDB/blob/main/Final_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Install Required Packages
!pip install -q pymongo sentence-transformers google-generativeai langchain langchain-google-genai langchain-community pypdf python-docx openpyxl pandas unstructured pillow langchain-text-splitters
!pip install -q duckduckgo-search beautifulsoup4 requests
!pip install -q rich

print("✅ Dependencies installed successfully.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.1/719.1 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
   ━━━━

In [7]:
pip install duckduckgo-search

In [2]:
# @title 2. Load API Keys & Imports
import os
import hashlib
import numpy as np
import uuid
import json
import requests
from typing import List, Dict, Any, Optional
from io import BytesIO

# ML & AI Imports
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from pymongo import MongoClient
from google.colab import userdata, files

# LangChain Imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    PyPDFLoader, Docx2txtLoader, TextLoader,
    UnstructuredExcelLoader, CSVLoader,
    UnstructuredMarkdownLoader, UnstructuredHTMLLoader
)
from langchain_core.documents import Document

# Web & Search Imports
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from PIL import Image

# UI Imports
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.table import Table
from rich import box

# Configuration
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    MONGODB_URI = userdata.get('MONGODB_URI')
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ API Keys loaded successfully!")
except Exception as e:
    print(f"❌ Error loading keys: {e}")
    print("Please set GEMINI_API_KEY and MONGODB_URI in Colab Secrets.")

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


✅ API Keys loaded successfully!


In [10]:
# @title 3. Define Advanced RAG Class (Expanded File Support & Free Web Search)
import os
import uuid
import hashlib
import requests
from io import BytesIO
from typing import List, Dict, Any
from PIL import Image

# GenAI & LangChain Imports
import google.generativeai as genai
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from pymongo import MongoClient
from duckduckgo_search import DDGS  # Required for free web search
from bs4 import BeautifulSoup

# Document Loaders
from langchain_community.document_loaders import (
    PyPDFLoader, Docx2txtLoader, CSVLoader, UnstructuredMarkdownLoader,
    UnstructuredHTMLLoader, TextLoader
)

class AdvancedRAGWithMemoryVisionWeb:
    def __init__(self, mongodb_uri: str, db_name: str = "rag", collection_name: str = "rag-collection"):
        """Initialize Advanced RAG with Memory, Vision, and Web capabilities"""

        # 1. Load Embedding Model
        print("🔄 Loading Embedding model (BAAI/bge-large-en-v1.5)...")
        self.embedding_model = SentenceTransformer('BAAI/bge-large-en-v1.5')
        self.embedding_dim = 1024

        # 2. Connect to MongoDB
        print("🔄 Connecting to MongoDB Atlas...")
        self.client = MongoClient(mongodb_uri)
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]
        self.memory_collection = self.db["conversation_memory"]

        # 3. Initialize Gemini
        self.model_name = 'gemini-2.5-flash-lite'
        print(f"🔄 Initializing {self.model_name}...")
        self.llm = genai.GenerativeModel(self.model_name)

        # 4. Text Splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, length_function=len
        )

        # 5. Session State
        self.session_id = None
        self.conversation_history = []

        print("✅ System Initialized!")

    # --- MEMORY ---
    def start_new_session(self):
        self.session_id = str(uuid.uuid4())
        self.conversation_history = []
        return self.session_id

    def add_to_memory(self, role: str, content: str):
        msg = {"role": role, "content": content, "timestamp": str(uuid.uuid4())}
        self.conversation_history.append(msg)
        if self.session_id:
            self.memory_collection.insert_one({**msg, "session_id": self.session_id})

    def get_context_str(self, last_n=5):
        return "\n".join([f"{m['role']}: {m['content']}" for m in self.conversation_history[-last_n:]])

    # --- VISION ---
    def analyze_uploaded_image(self, uploaded_file_dict, question="Describe this image"):
        try:
            filename = list(uploaded_file_dict.keys())[0]
            image_data = uploaded_file_dict[filename]
            image = Image.open(BytesIO(image_data))
            response = self.llm.generate_content([question, image])
            return response.text
        except Exception as e:
            return f"Error processing image: {str(e)}"

    # --- WEB (UPDATED FOR DUCKDUCKGO) ---
    def web_search_and_scrape(self, query: str, max_results=5):
        """
        Performs a free web search using DuckDuckGo and returns summaries.
        """
        try:
            print(f"🌐 Searching DuckDuckGo for: '{query}'...")
            context = []

            # Use the context manager to ensure the session is handled correctly
            with DDGS() as ddgs:
                # DuckDuckGo returns keys: 'title', 'href', 'body' (snippet)
                results = list(ddgs.text(query, max_results=max_results))

                if not results:
                    return "No web results found."

                for r in results:
                    # We use the 'body' provided by DDG directly.
                    # This is faster and avoids 403 Forbidden errors from scraping raw HTML.
                    source_info = (
                        f"Source: {r.get('title', 'Unknown')}\n"
                        f"URL: {r.get('href', '#')}\n"
                        f"Content Summary: {r.get('body', '')}"
                    )
                    context.append(source_info)

            return "\n\n".join(context)
        except Exception as e:
            print(f"❌ Web search error: {e}")
            return f"Web search failed: {str(e)}"

    # --- INGESTION ---
    def ingest_file(self, file_path: str, metadata: Dict = None):
        ext = os.path.splitext(file_path)[1].lower()

        # Define loaders for different types
        if ext == '.pdf':
            loader = PyPDFLoader(file_path)
        elif ext in ['.docx', '.doc']:
            loader = Docx2txtLoader(file_path)
        elif ext == '.csv':
            loader = CSVLoader(file_path)
        elif ext == '.md':
            try:
                loader = UnstructuredMarkdownLoader(file_path)
            except:
                loader = TextLoader(file_path)
        elif ext in ['.html', '.htm']:
            loader = UnstructuredHTMLLoader(file_path)
        elif ext in ['.txt', '.json', '.xml', '.py', '.js', '.java', '.c', '.cpp', '.yaml', '.yml', '.ini', '.log']:
            loader = TextLoader(file_path)
        else:
            print(f"⚠️ Skipping unsupported file: {os.path.basename(file_path)}")
            return 0

        try:
            docs = loader.load()
            if not docs: return 0

            chunks = self.text_splitter.split_documents(docs)

            new_chunks = 0
            for i, chunk in enumerate(chunks):
                chunk_hash = hashlib.sha256(chunk.page_content.encode()).hexdigest()

                if self.collection.find_one({"hash": chunk_hash}):
                    continue

                doc = {
                    "text": chunk.page_content,
                    "embedding": self.embedding_model.encode(chunk.page_content).tolist(),
                    "hash": chunk_hash,
                    "metadata": {**(metadata or {}), "chunk_index": i}
                }
                self.collection.insert_one(doc)
                new_chunks += 1
            return new_chunks
        except Exception as e:
            print(f"❌ Error ingesting {file_path}: {e}")
            return 0

    # --- RETRIEVAL & GENERATION ---
    def generate_answer(self, query: str, use_web=False):
        self.add_to_memory("user", query)

        ctx_sources = []
        web_content = ""

        # 1. Vector Search (Internal Docs)
        q_emb = self.embedding_model.encode(query).tolist()
        try:
            results = list(self.collection.aggregate([
                {"$vectorSearch": {
                    "index": "vector_index",
                    "path": "embedding",
                    "queryVector": q_emb,
                    "numCandidates": 50,
                    "limit": 3
                }},
                {"$project": {"_id": 0, "text": 1, "metadata": 1, "score": {"$meta": "vectorSearchScore"}}}
            ]))
            ctx_sources = results
        except Exception as e:
            print(f"⚠️ Vector Search failed: {e}")

        # 2. Web Search Logic
        # Trigger if explicitly requested OR keywords present OR internal docs are empty
        triggers = ["latest", "news", "current", "today", "who is", "what is"]
        if use_web or any(t in query.lower() for t in triggers):
            print("🔎 Triggering Web Search...")
            web_content = self.web_search_and_scrape(query)

        doc_text = "\n\n".join([f"[Doc Source: {r['metadata'].get('source')}]\n{r['text']}" for r in ctx_sources])
        history = self.get_context_str()

        # 3. Construct Prompt
        prompt = f"""
        You are an advanced AI assistant. Answer the user's question using the context provided.

        If the internal documents answer the question, prioritize them.
        If the web context is needed for recent events or general knowledge, use it.

        Conversation History:
        {history}

        Internal Document Context:
        {doc_text if doc_text else "No relevant internal documents found."}

        Web Search Context:
        {web_content if web_content else "No web search performed."}

        User Question: {query}
        """

        response = self.llm.generate_content(prompt)
        answer = response.text
        self.add_to_memory("assistant", answer)

        return {
            "answer": answer,
            "sources": ctx_sources,
            "web_used": bool(web_content)
        }

# Re-initialize
rag = AdvancedRAGWithMemoryVisionWeb(
    mongodb_uri=MONGODB_URI,
    db_name="rag",
    collection_name="rag-collection"
)

🔄 Loading Embedding model (BAAI/bge-large-en-v1.5)...
🔄 Connecting to MongoDB Atlas...
🔄 Initializing gemini-2.5-flash-lite...
✅ System Initialized!


In [11]:
# @title 4. Initialize & Create Indexes
# Initialize
rag = AdvancedRAGWithMemoryVisionWeb(
    mongodb_uri=MONGODB_URI,
    db_name="rag",
    collection_name="rag-collection"
)

# Create Hash Index for Deduplication (Runs once)
print("🔧 Ensuring hash index exists...")
try:
    rag.collection.create_index("hash", unique=True)
    print("✅ Hash index verified!")
except Exception as e:
    print(f"Note: {e}")

rag.start_new_session()

🔄 Loading Embedding model (BAAI/bge-large-en-v1.5)...
🔄 Connecting to MongoDB Atlas...
🔄 Initializing gemini-2.5-flash-lite...
✅ System Initialized!
🔧 Ensuring hash index exists...
✅ Hash index verified!


'afb2bd9d-987c-42bd-9f6d-9dd30534339b'

In [12]:
# @title 5. Upload & Ingest Documents (FIXED)
print("📤 Upload documents (PDF, DOCX, TXT, CSV)")
uploaded = files.upload()

total_chunks = 0

for filename, content in uploaded.items():
    print(f"Processing {filename}...")

    # Write to temp file
    temp_path = f"/tmp/{filename}"
    with open(temp_path, 'wb') as f:
        f.write(content)

    # Ingest
    chunks_added = rag.ingest_file(temp_path, metadata={"source": filename})

    # Safety Check: Ensure chunks_added is a number before adding
    if chunks_added is not None:
        total_chunks += chunks_added
    else:
        print(f"⚠️ Warning: No chunks returned for {filename}")

    # Cleanup
    if os.path.exists(temp_path):
        os.remove(temp_path)

print(f"\n✅ Ingestion Complete! Added {total_chunks} new chunks.")

📤 Upload documents (PDF, DOCX, TXT, CSV)


Saving Kethan VR A Comprehensive.md to Kethan VR A Comprehensive.md
Processing Kethan VR A Comprehensive.md...

✅ Ingestion Complete! Added 0 new chunks.


In [15]:
# @title 6. Interactive Chat Interface (Zero-Noise Mode)
import sys
import os
import warnings
import logging
from contextlib import contextmanager

# --- NUCLEAR OPTION: SUPPRESS ALL NOISE ---
# 1. Standard Python Warnings
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

# 2. Silence Third-Party Loggers
logging.getLogger("tornado.access").setLevel(logging.CRITICAL)
logging.getLogger("urllib3").setLevel(logging.CRITICAL)
logging.getLogger("duckduckgo_search").setLevel(logging.CRITICAL)

# 3. Context Manager to eat stderr during noisy operations
@contextmanager
def suppress_stderr():
    with open(os.devnull, "w") as devnull:
        old_stderr = sys.stderr
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stderr = old_stderr

from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.table import Table
from rich import box
from google.colab import files

console = Console()

console.print(Panel.fit(
    "[bold cyan]Advanced RAG System[/bold cyan]\n"
    "[dim]Commands:[/dim]\n"
    "• [green]Any question[/green]: Search docs + memory\n"
    "• [green]web: <query>[/green]: Force web search\n"
    "• [green]image[/green]: Upload and analyze image\n"
    "• [green]exit[/green]: Quit",
    title="🚀 Ready (Zero-Noise Mode)", border_style="cyan"
))

while True:
    try:
        # Use standard input() to avoid Colab recursion errors
        print("\n" + "─"*50)
        query = input("User 👤 (Type 'exit' to quit): ").strip()

        if query.lower() in ['exit', 'quit', 'q']:
            console.print("[bold red]👋 Goodbye![/bold red]")
            break

        if not query: continue

        # --- IMAGE MODE ---
        if query.lower() == 'image':
            console.print("[yellow]📤 Upload an image...[/yellow]")
            img_upload = files.upload()
            if img_upload:
                q_img = input("Question about image: ")
                with console.status("[bold green]Analyzing Image...[/bold green]"):
                    with suppress_stderr(): # SILENCE WARNINGS HERE
                        ans = rag.analyze_uploaded_image(img_upload, q_img)
                console.print(Panel(Markdown(ans), title="🖼️ Image Analysis", border_style="magenta"))
            continue

        # --- TEXT/WEB MODE ---
        use_web = False
        if query.startswith("web:"):
            use_web = True
            query = query.replace("web:", "").strip()

        with console.status("[bold green]Thinking...[/bold green]"):
            try:
                # WRAP GENERATION IN SILENCE
                # This eats the "duckduckgo" and "ipywidgets" red text
                with suppress_stderr():
                    if hasattr(rag, 'web_search_and_scrape'):
                         result = rag.generate_answer(query)
                    else:
                         result = rag.generate_answer(query)

                # Display Answer
                console.print(Panel(
                    Markdown(result['answer']),
                    title="🤖 AI Response",
                    border_style="green",
                    box=box.ROUNDED
                ))

                # Display Sources
                if result.get('sources'):
                    table = Table(title="📚 Sources Used", box=box.SIMPLE)
                    table.add_column("Score", style="cyan")
                    table.add_column("Source File", style="magenta")

                    for s in result['sources']:
                        table.add_row(
                            f"{s.get('score', 0):.2f}",
                            s['metadata'].get('source', 'unknown')
                        )
                    console.print(table)

                if result.get('web_used'):
                    console.print("[dim]🌐 Web content was used to answer this.[/dim]")

            except Exception as e:
                # We turn stderr back on for actual errors so you can see them
                console.print(f"[red]Generation Error: {e}[/red]")

    except KeyboardInterrupt:
        console.print("\n[bold red]👋 Goodbye![/bold red]")
        break
    except Exception as e:
        console.print(f"[red]System Error: {e}[/red]")

╭───── 🚀 Ready (Zero-Noise Mode) ─────╮
│ Advanced RAG System                  │
│ Commands:                            │
│ • Any question: Search docs + memory │
│ • web: <query>: Force web search     │
│ • image: Upload and analyze image    │
│ • exit: Quit                         │
╰──────────────────────────────────────╯


──────────────────────────────────────────────────
User 👤 (Type 'exit' to quit): who is modi


Output()

🔎 Triggering Web Search...

🌐 Searching DuckDuckGo for: 'who is modi'...

...

               📚 Sources Used               
                                             
  Score   Source File                        
 ─────────────────────────────────────────── 
  0.75    /tmp/Kethan VR A Comprehensive.md  
  0.74    /tmp/Kethan VR A Comprehensive.md  
  0.74    /tmp/Kethan VR A Comprehensive.md 

🌐 Web content was used to answer this.

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7970fe270520>



──────────────────────────────────────────────────


👋 Goodbye!